In [1]:
import requests as re
import os
from IPython.display import JSON  
import pandas as pd


In [2]:
client_id = os.environ["FOURSQUARE_CLIENT_ID"]
client_secret = os.environ["FOURSQUARE_CLIENT_SECRET"]

In [3]:
# location and version of the API
latitude = 51.0478
longitude= -114.0593
v = "20211104"
radius = 5000
query = "restaurants, bars"


In [4]:
url = "https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&radius={}&query={}&v={}".format(client_id,client_secret,latitude,longitude,radius,query,v)

In [6]:
res=re.get(url)
# print(res)
result = res.json()
JSON(result)

<IPython.core.display.JSON object>

In [7]:
import sqlite3
from sqlite3 import Error
   

In [8]:
def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")
    return connection 

In [9]:
connection = create_connection("downtown_calgary_POI.sqlite")

Connection to SQLite DB successful


In [10]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

In [11]:
#releveant part needed
venues = result['response']['venues']
# transform venues into a dataframe
df = pd.json_normalize(venues) 
df.head(2)
#remove come columns
df1= df.drop(columns=["categories","location.formattedAddress", "id","location.labeledLatLngs"])
df1.head(2)

,name,referralId,hasPerk,location.address,location.lat,location.lng,location.distance,location.postalCode,location.cc,location.city,location.state,location.country
0,Subway,v-1636115618,False,"520 3rd Ave SW, Suite 260",51.051100,-114.072098,968,T2P 0R3,CA,Calgary,AB,Canada
1,Subway,v-1636115618,False,"425 1st St SW, Unit 204",51.049175,-114.065675,471,T2P 3L8,CA,Calgary,AB,Canada


In [129]:
#create table in sql
df_sql = df1.to_sql('foursquare_venues', connection, if_exists='fail', index=True)

In [130]:
#Selecting records
def execute_read_query(connection, query):
    cursor= connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result=cursor.fetchall()
        return result
    except Error as e:
        print(f"The error '{e}' occurred")

In [131]:
four = "SELECT * from foursquare_venues"
four_ven = execute_read_query(connection, four)

#for ven in four_ven:
 #   print(ven)
    